# 摘要
此處目標為，將需要串接、整理之原始資料。整理成一份 excel。

以利後續，採用此份 excel 之內容，配合 "填值表" excel，將每一個數值填入 Nas_data 之中。

根據，reference資料夾的填值表，載入公式，根據公式，得到需要的變數種類。

最後，將每種變數清理好以後，進行串接。

In [3]:
import pandas
import os
import numpy
import traceback, re

In [2]:
formula_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "填值表.xlsx")
formula_data = pandas.read_excel(formula_data_path, sheet_name= "填值表")
# print(formula_data["公式(pandas)"])

# 使用正則表達式，將每一個公式之中使用的變數，抓入一個 list 之中。
import re

var_list = []
for formula in list(formula_data["公式(pandas)"]) : 
    try : 
        formula_split = formula.split("/")
        for var in formula_split :
            matches = re.findall(r"data\[\"(\S+)\"\]", var) 

            for matche in matches : 
                if matche not in var_list : 
                    var_list.append(matche)
    except Exception as e : 
        print(formula)
        print(traceback.print_exc())

# print(var_list) # 記得還有兩格沒有處理完。
for var in var_list : 
    print(var)

nan
None
nan
None
正式學籍在學學生數
職員、警衛及工友人數
專任教師數
日間學士及專科學生數
當學年度各學制總量內新生招生核定名額之實際註冊人數(C)
當學年度各學制總量內核定新生招生名額(A)
當學年度各學制新生保留入學資格人數(B)
當學年度各學制境外(新生)學生實際註冊人數(D)
日間學制學士班及專科申請延長修業年限之學生人數
當學年度2年級在學學生數(B)
前一學年度錄取1年級在學學生人數(A)
專任輔導人員數
專任教授人數
專任副教授人數
專任助理教授人數
編制外專任教師數
專任教師數(不含其他教師)
兼任教師數
日間學制學生數
日間學制專任教師數(含助教)
學雜費收入
總收入
負債總額
應付代管資產
資產總額
營運活動淨現金流量
流動負債
獲弱勢學生助學計畫補助之受惠學生人數
日間學制學士班及專科畢業生取得輔系資格人數
日間學制學士班及專科畢業生人數
日間學制學士班及專科畢業生取得雙主修資格人數
日間學制本國學生出國進修交流人數
日間學制正式學籍在學學生人數
境外學位生數
外籍專任教授人數
外籍專任副教授人數
外籍專任助理教授人數
日間學士及專科學年內退學人數
日間學制學士班及專科修讀輔系人次
日間學制學士班及專科修讀雙主修人次


Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_10752\2421629301.py", line 11, in <module>
    formula_split = formula.split("/")
AttributeError: 'float' object has no attribute 'split'
Traceback (most recent call last):
  File "C:\Users\user\AppData\Local\Temp\ipykernel_10752\2421629301.py", line 11, in <module>
    formula_split = formula.split("/")
AttributeError: 'float' object has no attribute 'split'


# 通用函數

In [3]:
# 透過更改分組變數，學校代碼、學校名稱，使不同 Data 可以被歸在一類。# sch_list 用於填入一個 需要串接的學校之學校代碼，第一個是學校代碼改變的依據。
def reindex_with_sch_function(data, sch_base, sch_rename_list) : # DataFrame, 學校名稱依據之學校代碼, 被改變之學校代碼。
    for sch in sch_rename_list : 
        data.loc[data["學校代碼"] == sch, "學校代碼"] = sch_base
    return data

# 根據學校代碼，從原始資料中刪除不需要的學校。
def del_row_by_sch_index(data, column_name, sch_index) : 
    data = data[data[column_name] != sch_index]
    return data

# 以最新的年份，為學校統一命名。
def rename_sch_name_by_sch_index(data, sch_index) :
    sch_index_data = data[data["學校代碼"] == sch_index].copy()
    sch_name = sch_index_data["學校名稱"].values[-1]
    sch_index_data["學校名稱"] = sch_name
    data[data["學校代碼"] == sch_index] = sch_index_data
    return data

# 將所有資料的 學校類別 都更改為最新學年度。
def retype_by_sch_year(df) : 
    for sch_index in list(dict.fromkeys(df["學校代碼"])) :
        sch_data = df[df["學校代碼"] == sch_index]
        sch_data.loc[sch_data.index, "學校類別"] = sch_data.sort_values("學年度")["學校類別"].values[-1]
        df[df["學校代碼"] == sch_index] = sch_data
    return df

# 將所有學校名稱相同的學校，其學校代碼改成最新學年度。
def reindex_by_sch_year(df) :
    for sch_name in list(dict.fromkeys(df["學校名稱"])) : 
        sch_data = df[df["學校名稱"] == sch_name]
        sch_data.loc[sch_data.index, "學校代碼"] = sch_data.sort_values("學年度")["學校代碼"].values[-1]
        df[df["學校名稱"] == sch_name] = sch_data
    return df

# 將數值欄位，資料型別盡可能轉為浮點數。本專案，所有的data，都是學校代碼在第一列，學年度在第二列，其餘為各項數值。
def retype_with_float(df) : 
    for column in df.columns[1:] : 
        try : 
            df[column] = df[column].astype(float)
        except : 
            print(column)
    return df

# 變數命名規則

如果這筆資料，不需要與其他資料串接，則命名為，

df_mered_???

以 "XXX.xlsx 統計" 為例。

df_mered_stu_12_2。mered 對應的是 「已串接」， stu 對應的是「學」字，「12_2」對應的是編號。

如果這筆資料，需要與其他資料串接，才能形成用於計算的變數，則命名為，

df_mer_???

以 "XXX.xlsx 統計" 為例。

df_mer_3。其中 用 mer 對應的是 「需串接」。

而串接完成之檔案，會以 df_mered_???_plus_??? 命名。如 df_mer_3 + df_mer_4 = df_mered_3_plus_4。

如此命名的理由在於，如果要一次性清理、串接所有的 data，並於最後輸出成 excel ，能夠根據 變數名稱

對所有符合該變數名稱之變數，直接檔案輸出。如 僅選取，df_mered，為開頭的變數，做輸出。

# 變數取得、串接
這裡需要將，每一筆原始資料，整理至，僅留下學校代碼、學年度、需要的數值變數。其餘皆不保留

並且資料型別為，學校代碼 : object、學年度 : float or int、需要的數值 : float。

## 專任教師數、專任教授人數、專任副教授人數、專任助理教授人數
資料來源 : 1.1專任教師數、教1-2.專任教師數。

### df_mer_1_1

In [4]:
folder_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "source"), "1-1.專任教師數")
folder_generator = os.walk(folder_path)
data_path_list = []
for dirs, subdirs, files in folder_generator : 
    for file in files : 
        data_path_list.append(os.path.join(dirs, file))

df_mer_1_1 = pandas.read_excel(data_path_list[0], header=2, dtype= {"學校代碼" : str})

df_mer_1_1 = df_mer_1_1.rename(columns= {"Unnamed: 1" : "學年度", "總計" : "專任教師數"})
df_mer_1_1 = df_mer_1_1[df_mer_1_1["學校名稱"].notnull()]
df_mer_1_1["專任教授人數"] = df_mer_1_1["男.1"] + df_mer_1_1["女.1"]
df_mer_1_1["專任副教授人數"] = df_mer_1_1["男.2"] + df_mer_1_1["女.2"]
df_mer_1_1["專任助理教授人數"] = df_mer_1_1["男.3"] + df_mer_1_1["女.3"]
df_mer_1_1 = df_mer_1_1[["學校代碼", "學年度", "專任教師數", "專任教授人數", "專任副教授人數", "專任助理教授人數"]]

### df_mer_tea_1_2

In [5]:
folder_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "source"), "教1-2.專任教師數-以「校」統計")
folder_generator = os.walk(folder_path)
data_path_list = []
for dirs, subdirs, files in folder_generator : 
    for file in files : 
        data_path_list.append(os.path.join(dirs, file))

df_mer_tea_3_2 = pandas.read_excel(data_path_list[0], header=2, dtype= {"學校統計處代碼" : str})
df_mer_tea_3_2 = df_mer_tea_3_2.rename(columns = {"學校統計處代碼" : "學校代碼", "專任教師數-教師總數總計" : "專任教師數"})
df_mer_tea_3_2 = df_mer_tea_3_2[df_mer_tea_3_2["設立別"].notnull()]
df_mer_tea_3_2["專任教授人數"] = df_mer_tea_3_2["專任教師數-教授男"] + df_mer_tea_3_2["專任教師數-教授女"]
df_mer_tea_3_2["專任副教授人數"] = df_mer_tea_3_2["專任教師數-副教授男"] + df_mer_tea_3_2["專任教師數-副教授女"]
df_mer_tea_3_2["專任助理教授人數"] = df_mer_tea_3_2["專任教師數-助理教授男"] + df_mer_tea_3_2["專任教師數-助理教授女"]
df_mer_tea_3_2 = df_mer_tea_3_2[["學校代碼", "學年度", "專任教師數", "專任教授人數", "專任副教授人數", "專任助理教授人數"]]

### df_mered_1_1_plus_tea_1_2

In [6]:
df_mered_1_1_plus_tea_1_2 = pandas.concat([df_mer_1_1, df_mer_tea_3_2])

## 外籍專任教師數、外籍專任教授人數、外籍專任副教授人數、外籍專任助理教授人數
資料來源 : 【大專校院校務資訊公開平臺】之「2.外籍專任教師數」（101-105學年度）、「教3-2.外籍專任教師數-以「校」統計」（106-111學年度）

### df_mer_2

In [7]:
folder_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "source"), "2.外籍專任教師數")
folder_generator = os.walk(folder_path)
data_path_list = []
for dirs, subdirs, files in folder_generator : 
    for file in files : 
        data_path_list.append(os.path.join(dirs, file))

df_mer_2 = pandas.read_excel(data_path_list[0], header=2, dtype= {"學校代碼" : str})
df_mer_2 = df_mer_2.rename(columns= {"Unnamed: 1" : "學年度", "總計" : "外籍專任教師數"})
df_mer_2 = df_mer_2[df_mer_2["學校名稱"].notnull()]
df_mer_2["外籍專任教授人數"] = df_mer_2["男.1"] + df_mer_2["女.1"]
df_mer_2["外籍專任副教授人數"] = df_mer_2["男.2"] + df_mer_2["女.2"]
df_mer_2["外籍專任助理教授人數"] = df_mer_2["男.3"] + df_mer_2["女.3"]
df_mer_2 = df_mer_2[["學校代碼", "學年度", "外籍專任教師數", "外籍專任教授人數", "外籍專任副教授人數", "外籍專任助理教授人數"]]

### df_mer_tea_3_2

In [8]:
folder_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "source"), "教3-2.外籍專任教師數-以「校」統計")
folder_generator = os.walk(folder_path)
data_path_list = []
for dirs, subdirs, files in folder_generator : 
    for file in files : 
        data_path_list.append(os.path.join(dirs, file))

df_mer_tea_3_2 = pandas.read_excel(data_path_list[0], header = 2, dtype= {"學校統計處代碼" : str})
df_mer_tea_3_2 = df_mer_tea_3_2.rename(columns = {"學校統計處代碼" : "學校代碼", "外籍專任教師數-教師總數總計" : "外籍專任教師數"})
df_mer_tea_3_2 = df_mer_tea_3_2[df_mer_tea_3_2["設立別"].notnull()]
df_mer_tea_3_2["外籍專任教授人數"] = df_mer_tea_3_2["外籍專任教師數-教授男"] + df_mer_tea_3_2["外籍專任教師數-教授女"]
df_mer_tea_3_2["外籍專任副教授人數"] = df_mer_tea_3_2["外籍專任教師數-副教授男"] + df_mer_tea_3_2["外籍專任教師數-副教授女"]
df_mer_tea_3_2["外籍專任助理教授人數"] = df_mer_tea_3_2["外籍專任教師數-助理教授男"] + df_mer_tea_3_2["外籍專任教師數-助理教授女"]
df_mer_tea_3_2 = df_mer_tea_3_2[["學校代碼", "學年度", "外籍專任教師數", "外籍專任教授人數", "外籍專任副教授人數", "外籍專任助理教授人數"]]


### df_mered_2_plus_tea_3_2

In [9]:
df_mered_2_plus_tea_3_2 = pandas.concat([df_mer_2, df_mer_tea_3_2])

In [10]:
df_mered_2_plus_tea_3_2

,學校代碼,學年度,外籍專任教師數,外籍專任教授人數,外籍專任副教授人數,外籍專任助理教授人數
0,0001,101,1.0,0.0,0.0,1.0
1,0001,101,6.0,0.0,3.0,1.0
2,0001,101,3.0,1.0,0.0,0.0
3,0001,101,2.0,0.0,0.0,1.0
4,0001,101,4.0,1.0,0.0,1.0
...,...,...,...,...,...,...
729,1R04,111,1.0,0.0,0.0,1.0
730,1R05,111,2.0,0.0,1.0,1.0
731,1R07,111,1.0,0.0,0.0,1.0
732,1R08,111,3.0,1.0,0.0,2.0


### df_mered_tea_8

In [11]:
folder_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "source"), "教8.專、兼任輔導人員數-以「校」統計")
folder_generator = os.walk(folder_path)
data_path_list = []
for dirs, subdirs, files in folder_generator : 
    for file in files : 
        data_path_list.append(os.path.join(dirs, file))

df_mered_tea_8 = pandas.read_excel(data_path_list[0], header=2, dtype= {"學校統計處代碼" : str})
df_mered_tea_8 = df_mered_tea_8.rename(columns= {"學校統計處代碼" : "學校代碼", "輔導人員數-專任" : "專任輔導人員數"})
df_mered_tea_8 = df_mered_tea_8[["學校代碼", "學期", "學年度", "專任輔導人員數"]]
df_mered_tea_8 = df_mered_tea_8[df_mered_tea_8["學校代碼"].notnull()]
df_mered_tea_8 = df_mered_tea_8[df_mered_tea_8["學期"] == 1]

In [12]:
df_mered_tea_8[(df_mered_tea_8["學校代碼"] == "1043") & (df_mered_tea_8["學年度"] == 108)]

,學校代碼,學期,學年度,專任輔導人員數
868,1043,1.0,108,6.0


# Data 清理、整理。
此處，將所有以 df_mered 開頭之變數 (所有 dataframe)，全部清理、整理。


## 抓出所有以 df_mered 開頭之變數，用於下一段迭代。

In [43]:
# 抓取本專案所有的變數。備註 : 如果出現不想出現的東西，可能有兩種情況。1. 列表推導式有問題。2. 變數出現的順序讓 globals() 抓到多餘的變數，這時候請從新啟動。
globals_vars = globals()

# 透過列表推導式，抓出 df_mered 開頭之變數，並組成一個 list。
df_mered_vars = [var for var in globals_vars if  var.startswith("df_mered") and not var.endswith("path")]



In [44]:
df_mered_vars

['df_mered_1_1_plus_tea_1_2',
 'df_mered_2_plus_tea_3_2',
 'df_mered_tea_8',
 'df_mered_tea_11',
 'df_mered_tea_2_2',
 'df_mered_12_plus_tea_5',
 'df_mered_sch_3_plus_sch_10',
 'df_mered_stu_2_2',
 'df_mered_tea_5_plus_religion',
 'df_mered_stu_12_2',
 'df_mered_3_plus_stu_4',
 'df_mered_1_plus_stu_1_2',
 'df_mered_15_plus_stu_14_2',
 'df_mered_stu_16']

## 資料型別轉換

In [45]:
for var in df_mered_vars : 
    # 數值變數轉浮點數
    print(var)
    globals_vars[var] = retype_with_float(globals_vars[var]) # 正常來說，這裡不應該 print 出，除了 data 名稱以外的東西。#學年度也一起轉為 float。
    # 學校代碼轉字串。
    globals_vars[var]["學校代碼"] = globals_vars[var]["學校代碼"].astype(str)
    

df_mered_1_1_plus_tea_1_2
df_mered_2_plus_tea_3_2
df_mered_tea_8
df_mered_tea_11
df_mered_tea_2_2
df_mered_12_plus_tea_5
df_mered_sch_3_plus_sch_10
df_mered_stu_2_2
df_mered_tea_5_plus_religion
df_mered_stu_12_2
df_mered_3_plus_stu_4
df_mered_1_plus_stu_1_2
df_mered_15_plus_stu_14_2
df_mered_stu_16


## 資料清理、整理

In [46]:
# 這裡當有其他學校停辦時，需要再新增。換言之，實作上需要準備一份，excel 存放這類的資料，讓程式可以讀取。
model_3_sch_text = "# 稻江科技暨管理學院#1176。# 蘭陽技術學院#1182。# 南榮技術學院#1181。# 南榮學校財團法人南榮科技大學#1074。# 永達技術學院#1154。# 高鳳數位內容學院#1190。# 亞太學校財團法人亞太創意技術學院#1189。#臺北基督學院#1R03。#臺灣觀光學院#1192。#高美醫護管理專科學校#1288。"
model_3_sch_list = []

model_3_sch_text_split = model_3_sch_text.split("。")
model_3_sch_text_split = model_3_sch_text_split[: -1]
# print(model_3_sch_text_split)
for element in model_3_sch_text_split :
    element_split = element.split("#")
    model_3_sch_list.append({"sch_name" : element_split[1], "sch_index" : element_split[2]})

# 資料串接。
# 年數大者，在前面
reindex_sch_text = "#台北海洋科技大學#1083#1194。#崇仁醫護管理專科學校#1293#1290。# 康寧大學#1079#1281#1059。# 國立陽明交通大學#0007#0016。# 國立清華大學#0002#0038。# 臺北市立大學#3002#3102#3001。# 國立屏東大學#0052#0136#0040。# 國立高雄科技大學#0053#0034#0026#0027。# 法鼓學校財團法人法鼓文理學院#1196#1R01。# 台北海洋學校財團法人台北海洋科技大學#1083#1197。# 華夏學校財團法人華夏科技大學#1076#1191。#國立臺北商業學院#0051#0142。#文藻外語大學#1075#1147。# 慈濟科技大學#1077#1150。# 致理科技大學#1078#1164。# 亞東學校財團法人亞東科技大學#1084#1166。# 宏國學校財團法人宏國德霖科技大學#1080#1179。# 東方學校財團法人東方設計大學#1081#1184。# 崇右學校財團法人崇右影藝科技大學#1082#1187。"

# 創建模式一，函數使用，list。
reindex_sch_list = []
reindex_sch_list_step_1 = reindex_sch_text.split("。")
reindex_sch_list_step_1 = reindex_sch_list_step_1[:-1] # 刪除多的一個，""。
# print(model_1_sch_list_step_1)
for element in reindex_sch_list_step_1 : 
    element_split = element.split("#")
    reindex_sch_list_element = {"sch_name" : element_split[1], "sch_base" : element_split[2], "sch_rename_list" : element_split[3:]}
    reindex_sch_list.append(reindex_sch_list_element)


# 使用 for - loop 整理每一個 data_mered 之變數。
for var in df_mered_vars : 

    # 刪除多餘的學校
    for element_m_3 in model_3_sch_list : 
        globals_vars[var] = del_row_by_sch_index(globals_vars[var], "學校代碼", element_m_3["sch_index"])


    # 資料串接
    for elment_re in reindex_sch_list : 
        try : 
            globals_vars[var] = reindex_with_sch_function(globals_vars[var], elment_re["sch_base"], elment_re["sch_rename_list"]) 
            
        except : 
            print(var, element["sch_base"], element["sch_rename_list"]) # 空，意味著這個方法，可以適用於目前給的所有檔案。

    


## 資料計算

In [47]:
error = []

data_output = []
for var in df_mered_vars : 
    data = globals_vars[var].copy()
    for column in data.columns[2:] : # 以每一個數值欄位，作為一份 data 儲存。舉例而言，df_mered_stu_12_2_data 有 4 格數值欄位，則會被轉為四份 Data。並存入 data_output。
        # print(column)
        data_output_element = []
        for sch_index in list(dict.fromkeys(data["學校代碼"])) : 
            sch_index_data = data[data["學校代碼"] == sch_index]
            for sch_year in list(dict.fromkeys(data["學年度"])): 
                try : 
                    sch_year_data = sch_index_data[sch_index_data["學年度"] == sch_year] # 某校、某年之數據。
                    if len(sch_year_data[column]) != 0 : # sch_year_data[column] 不是空的，以及，不是全都是遺漏值。
                        if sch_year_data[column].isnull().sum() != len(sch_year_data[column]) : 
                            var_sum = sch_year_data[column].sum() # 可能是，明明已經轉換為 float 了，但問題是，有部分的非數值型別混在裡面。
                        else : 
                            var_sum = numpy.nan
                    else : 
                        var_sum = numpy.nan
                    data_element = {"學校代碼" : sch_index, "學年度" : sch_year, column : var_sum}
                    data_output_element.append(data_element)
                except Exception as e : 
                    if traceback.print_exc() not in error:
                        error.append(traceback.print_exc())

        data_output_element = pandas.DataFrame(data_output_element)
        data_output.append(data_output_element)


print(error)

[]


## 資料串接

In [48]:
# print(df_mered_vars)
# print(df_mered_stu_12_2_data)

merged_data_left = data_output[0]
for data in data_output[1:] : 
    merged_data_left = pandas.merge(merged_data_left, data, on=["學校代碼", "學年度"], how='outer')

# 學年度，由小到大排列。
merged_data_left = merged_data_left.sort_values("學年度")

In [49]:
# 不是很清楚為何會有學年度為 NaN 的行數出現。並且是多出來的，僅在 1R04、1R07、1R08 出現。
merged_data_left[merged_data_left["學校代碼"] == "1R04"]



,學校代碼,學年度,專任教師數,專任教授人數,專任副教授人數,專任助理教授人數,外籍專任教師數,外籍專任教授人數,外籍專任副教授人數,外籍專任助理教授人數,...,當學年度各學制新生保留入學資格人數(B),當學年度各學制總量內新生招生核定名額之實際註冊人數(C),當學年度各學制境外(新生)學生實際註冊人數(D),日間學制學士班及專科申請延長修業年限之學生人數,正式學籍在學學生數,日間學士及專科學生數,日間學制正式學籍在學學生人數,日間學士及專科學年內退學人數,前一學年度錄取1年級在學學生人數(A),當學年度2年級在學學生數(B)
1639,1R04,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1640,1R04,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1641,1R04,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1642,1R04,104.0,7.0,3.0,2.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.0,NaN,12.0,NaN,NaN,NaN
1643,1R04,105.0,9.0,0.0,3.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,27.0,NaN,27.0,NaN,NaN,NaN
1644,1R04,106.0,9.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,...,1.0,40.0,0.0,NaN,72.0,40.0,72.0,2.0,NaN,NaN
1645,1R04,107.0,9.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN,...,0.0,30.0,0.0,NaN,128.0,61.0,98.0,6.0,40.0,30.0
1646,1R04,108.0,11.0,2.0,3.0,5.0,1.0,0.0,0.0,1.0,...,0.0,25.0,1.0,NaN,171.0,75.0,109.0,9.0,31.0,22.0
1647,1R04,109.0,12.0,2.0,3.0,7.0,1.0,0.0,0.0,1.0,...,0.0,24.0,3.0,NaN,178.0,88.0,117.0,23.0,25.0,19.0
1648,1R04,110.0,10.0,1.0,3.0,6.0,1.0,0.0,0.0,1.0,...,0.0,23.0,0.0,1.0,150.0,69.0,94.0,20.0,24.0,10.0


# 變數輸出
將每一個整理、串接好的 data，存入一個 excel。

In [50]:
output_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step3_data_base.xlsx")
merged_data_left.to_excel(output_path, index = False)

# 測試區

In [51]:
rr = pandas.DataFrame({
    "A" : [1, numpy.nan, 2],
    "B" : [1, 2, numpy.nan]
})
print(rr["A"].values)
print(rr["A"].values[0] + rr["B"].values[0]) # 遺漏值 + 非遺漏值 = 遺漏值。


print(list(dict(rr["A"]))) 
# 該欄位有遺漏值，但有其餘數值，sum()僅會計算，數值部分的加總。
# 該欄位全部都是遺漏值，sum()，會輸出 0。但這一個應該要輸出 numpy.nan。
# 該欄位沒有東西，sum()，會輸出 0。但實際上，這一格應該要輸出 numpy.nan。

# 如此一來，無法確定，究竟目前在 Nas_base中，某校某年，顯示為 0 的數值，究竟是來自於一個空集合，還是一個真的具有 0 數值的欄位。
# 這個差別會影響到，後續要將具有部分零數值的學校，其欄位，設定為 0 時，無法分辨其到底是否為真的具有 0 數值欄位。

# 上述問題，透過在資料計算區塊，透過 if 判別式，區辨資料計算的來源情況，預期可以解決。

[ 1. nan  2.]
2.0
[0, 1, 2]
